# 4.蒙特卡洛方法（Monte-Carlo method）
**蒙特卡洛方法**（Monte-Carlo methods）也被称为统计模拟方法，是一种基于概率统计的数值计算方法
用蒙特卡洛方法的好处在于不需要知道 MDP 的状态转移函数和奖励函数，就可以近似估计价值函数，并且采样数越多越准确

导入相关库

In [62]:
import numpy as np

直观理解**蒙特卡洛方法**：
![用蒙特卡洛方法估计圆的面积](images/用蒙特卡洛方法估计圆的面积.png)

用蒙特卡洛方法来估计一个策略在一个马尔可夫决策过程中的状态价值函数，公式如下：
$$V^\pi(s)=\mathbb{E}_\pi[G_t|S_t=s]\approx\frac{1}{N}\sum_{i=1}^NG_t^{(i)}$$
在一条序列中，可能不会出现状态s，也可能只出现一次或多次，蒙特卡洛价值估计方法会在该状态每一次出现时计算它的回报：
- 更新状态s的计数器:$N(s)\leftarrow N(s)+1$
- 更新状态s的总回报:$M(s)\leftarrow M(s)+G_t$
- 每一个状态的价值被估计为回报的平均值:$V(s)=M(s)/N(s)$

根据大数定律，当$N(s)\to\infty\text{,}V(s)\to V^{\pi}(s)$
- 采用增量式更新:$V(s)\leftarrow V(s)+\frac{1}{N(s)}(G(s)-V(s))$

以下，用代码展示**蒙特卡洛方法**

复用上一节的简单马尔可夫决策过程:
![马尔可夫决策过程的一个简单例子](images/马尔可夫决策过程的一个简单例子.png)

In [63]:
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态集合
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作集合
# 状态转移函数
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}
# 奖励函数
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}
gamma = 0.5  # 折扣因子
MDP = (S, A, P, R, gamma)

# 策略1,随机策略
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}
# 策略2
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}

In [64]:
# 把输入的两个字符串通过“-”连接，连接状态与动作
def join(str1, str2):
    return str1 + '-' + str2

In [65]:
def sample(MDP, Pi, timestep_max, number):
    """ 采样函数：
    Pi：策略
    timestep_max：限制每采样最长步数
    number：以固定状态s开始，总共采样序列数
    """
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = []
        timestep = 0
        s = S[np.random.randint(4)]  # 随机选择一个除s5以外的状态s作为起点
        # 当前状态为终止状态或者时间步太长时,一次采样结束
        while s != "s5" and timestep <= timestep_max:
            timestep += 1
            rand, temp = np.random.rand(), 0  # 随机参数
            # 在状态s下根据策略选择动作
            for a_opt in A:
                temp += Pi.get(join(s, a_opt), 0)  # 非x即y
                if temp > rand:
                    a = a_opt  # 确定动作
                    r = R.get(join(s, a), 0)  # 获取奖励
                    break
            rand, temp = np.random.rand(), 0  # 随机参数
            # 根据状态转移概率得到下一个状态s_next
            for s_opt in S:
                temp += P.get(join(join(s, a), s_opt), 0)
                if temp > rand:
                    s_next = s_opt
                    break
            episode.append((s, a, r, s_next))  # 把（s,a,r,s_next）元组放入序列中
            s = s_next  # s_next变成当前状态,开始接下来的循环
        episodes.append(episode)
    return episodes

In [66]:
# 采样5次,每个序列最长不超过20步
episodes = sample(MDP, Pi_1, 20, 5)
print('第一条序列\n', episodes[0])
print('第二条序列\n', episodes[1])
print('第五条序列\n', episodes[4])

第一条序列
 [('s2', '前往s3', -2, 's3'), ('s3', '前往s5', 0, 's5')]
第二条序列
 [('s3', '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]
第五条序列
 [('s3', '前往s5', 0, 's5')]


In [67]:
# 对所有采样序列计算所有状态的价值（增量法）
def MC(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        for i in range(len(episode) - 1, -1, -1):  # 一个序列从后往前计算
            (s, a, r, s_next) = episode[i]
            G = r + gamma * G
            N[s] = N[s] + 1
            V[s] = V[s] + (G - V[s]) / N[s]

In [68]:
# 采样1000次,每个序列最长不超过20步
episodes = sample(MDP, Pi_1, 20, 1000)
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
MC(episodes, V, N, gamma)
print("使用蒙特卡洛方法计算MDP的状态价值为\n", V)

使用蒙特卡洛方法计算MDP的状态价值为
 {'s1': -1.2141993558934854, 's2': -1.6588677480829939, 's3': 0.5242593857923498, 's4': 6.219433362216237, 's5': 0}


上节 **MRP 解析解** ：
![MRP解析解](images/MRP解析解.png)

可以看到用 **蒙特卡洛方法** 估计得到的状态价值和用上节 **MRP 解析解** 得到的状态价值是很接近的（注意需采样较多的序列）